# Use Plain Attributes Rather than Get and Set

It is rather common to write a class with get and set attributes. Consider the example below which has a `DistanceRun` class containing two methods to set and get the distance run in kilometres .

In [1]:
class DistanceRun:
    
    def __init__(self, distance_km):
        self._distance_km = distance_km
        
    def get_distance(self):
        return self._distance_km
        
    def set_distance(self, distance):
        self._distance_km = distance

In [2]:
d = DistanceRun(10)
d.get_distance()

10

If we want to change the distance run we could do something like this:

In [3]:
d.set_distance(d.get_distance() + 5)
d.get_distance()

15

This is rather ugly and in Python you never really need to explicitly define getter and setter methods. In this case, the distance run should be set as public attribute:

In [4]:
class DistanceRun:
    
    def __init__(self, distance_km):
        self.distance_km = distance_km

In [5]:
d = DistanceRun(10)
d.distance_km

10

Taking this approach makes it much clearer when increasing the distance:

In [6]:
d.distance_km +=5
d.distance_km

15

If you need special behaviour you can take advantage of the `@property` decorator and the corresponding `setter` attribute. I’ve updated the `DistanceRun` class below to now also calculate the distance run in miles:

In [7]:
class DistanceRun:
    
    def __init__(self, distance_km):
        self._distance_km = distance_km
        self.distance_miles = distance_km / 1.6
    
    @property
    def distance_km(self):
        return self._distance_km
    
    @distance_km.setter
    def distance_km(self, distance_km):
        self._distance_km = distance_km
        self.distance_miles = self._distance_km / 1.6

In [8]:
d = DistanceRun(10)

In [9]:
d.distance_km

10

In [10]:
d.distance_miles

6.25

In [11]:
d.distance_km = 50

In [12]:
d.distance_km

50

In [13]:
d.distance_miles

31.25

Setting a new distance run will update the distance run in miles too!

Now, because we have a `setter` property we also validate/check the values passed to the class. Here, I check to ensure the distance run is non-negative:

In [14]:
class DistanceRun:
    
    def __init__(self, distance_km):
        self._distance_km = distance_km
        self.distance_miles = distance_km / 1.6
    
    @property
    def distance_km(self):
        return self._distance_km
    
    @distance_km.setter
    def distance_km(self, distance_km):
        if distance_km < 0:
            raise ValueError('{} distance run must be >= 0'.format(distance_km))
        self._distance_km = distance_km
        self.distance_miles = self._distance_km / 1.6

In [15]:
d = DistanceRun(10)

In [16]:
d.distance_km

10

In [17]:
d.distance_km = -100

ValueError: -100 distance run must be >= 0

But the exception won't be raised in the constructor:

In [18]:
d = DistanceRun(-99)

In [19]:
d.distance_km

-99

For this to happen, we will need to adjust the class slightly. The `DistanceRun` class now inherits from the `Distance` class (For this to work correctly, the name of the `getter` and `setter` methods must match the intended property name).

In [20]:
class Distance:
    def __init__(self, distance_km):
        self.distance_km = distance_km

class DistanceRun(Distance):
    
    def __init__(self, distance_km):
        super().__init__(distance_km)
        self.distance_miles = distance_km / 1.6
    
    @property
    def distance_km(self):
        return self._distance_km
    
    @distance_km.setter
    def distance_km(self, distance_km):
        if distance_km < 0:
            raise ValueError('{} distance run must be >= 0'.format(distance_km))
        self._distance_km = distance_km
        self.distance_miles = self._distance_km / 1.6

In [21]:
dist = DistanceRun(-99) # Rasies error now

ValueError: -99 distance run must be >= 0

This works because setting the `DistanceRun.__init__` calls `Distance.__init__` which assigns ` self.distance_km = -99`. The assignment actually causes ` @distance_km.setter` to be called which will raise the error.

We still have the functionality of check/validating the distance when trying to re-assign the distance:

In [22]:
d = DistanceRun(10)

In [23]:
d.distance_km = -5

ValueError: -5 distance run must be >= 0